In [1]:
%load_ext autoreload
%autoreload 2

In [21]:
import open_clip
from IPython.display import clear_output
import torch
from PIL import Image
import os
import sys
sys.path.append("../model/")
from dataset import DeepFakeDatasetFastLoad, transform_torch
import numpy as np
from tqdm import tqdm

In [13]:
device = "cpu"
model, _, preprocess = open_clip.create_model_and_transforms('hf-hub:laion/CLIP-ViT-L-14-DataComp.XL-s13B-b90K',device=device)
model.eval()
clear_output()

In [10]:
data = DeepFakeDatasetFastLoad("/data4/saland/data/df_34000_V2.pt",
                               remove_blacklisted_gen=True)

data.features.shape

/home/lsaland/micromamba/envs/clip/lib/python3.11/site-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)
filtering: 100%|██████████| 12/12 [00:00<00:00, 37.95it/s]


torch.Size([30000, 768])

In [23]:
path_to_flickr = "/data4/saland/data/2k_real_2k_fake/Flickr/"
flickr_files = os.listdir(path_to_flickr)
transform = lambda img : preprocess(Image.fromarray(transform_torch(image=np.asarray(img.convert("RGB")))["image"]))

imgs_CPU = []
imgs_GPU = []
for file in tqdm(flickr_files):
    img = Image.open(path_to_flickr + file)
    pre = transform(img).unsqueeze(0)
    imgs_CPU.append(pre)
    imgs_GPU.append(pre.to("cuda:0"))

100%|██████████| 1000/1000 [02:10<00:00,  7.69it/s]


In [24]:
%%timeit
model.encode_image(torch.cat(imgs_CPU,dim=0))

In [ ]:
model.to("cuda:0")

In [ ]:
%%timeit
model.to("cuda:0")